<h1 align='center'>Downloading, Pre-processing, and Simplifying Shodan.io Raw Data<br>Applied to Investigate Mikrotik Devices</h1>

<h1 align='center'>========================================================</h1>

## Connecting to Shodan CLI
[https://shodan.io](https://shodan.io)

In [4]:
shodankey = "dcv9HqJGyaGEWa1t5lDiDXrI0CrG3fTp" #PROCEDURE: Login in shodan.io -> My account -> API key

In [2]:
!shodan init $shodankey

Successfully initialized


<h1 align='center'>========================================================</h1>

## Making queries using Shodan CLI

In [2]:
!shodan count mikrotik

1693812


In [3]:
!shodan count product:mikrotik

4127099


In [4]:
!shodan count product:mikrotiksmb

1458


In [5]:
!shodan count vendor: mikrotik

1178172


In [14]:
!shodan count port:2000

4543590


In [15]:
!shodan count port:1723

4548614


**NOTE:** the main problem is that we don't know wheter Shodan is classifying and retrieving the most accurated number of Mikrotik devices. We don't know which of the queries we should rely on. **Take away: better to get the raw data and classify ourselves!**

<h1 align='center'>========================================================</h1>

## Downloading all the results using query "mikrotik" (via CLI) [LIMIT 1M]
**NOTE:** 1 query credit = 100 results

**NOTE:** this approach is limited to 1M results. 

In [ ]:
!shodan download --limit -1 shodan_output "mikrotik"

**NOTE:** Shodan takes around 2 weeks to perform the port-scan in the entire Internet, therefore We also need to download 2 weeks in advance, merge and analyse!!!!

In [ ]:
!shodan download --limit -1 shodan_mikrotik_older "mikrotik before:04/07/2019"

**NOTE:** Following a printscreen while we've tried to download more than 1M entries

![fig](./figs/evidence_less_than_1m_records_shodan.png)

<h1 align='center'>========================================================</h1>

## Downloading raw data via an enterprise account [UNLIMITED DATA]
To overcome the limitation of retrieving only 1M of entries, we've contacted support@shodan.io asking for the possibility to perform 'Bulk Downloads'. It is a functionality only available to 'Enterprise Accounts' (https://enterprise.shodan.io/product-comparison). Well, they gave us a temporary enterprise account accessible at [https://enterprise.shodan.io](https://enterprise.shodan.io). Thanks John Matherly (twitter: @achillean) 

## Retrieving the list of datasets available 

In [3]:
!shodan data list

Traceback (most recent call last):
  File "/usr/local/bin/shodan", line 11, in <module>
    load_entry_point('shodan==1.13.0', 'console_scripts', 'shodan')()
  File "/usr/local/lib/python3.7/site-packages/click/core.py", line 764, in __call__
    return self.main(*args, **kwargs)
  File "/usr/local/lib/python3.7/site-packages/click/core.py", line 717, in main
    rv = self.invoke(ctx)
  File "/usr/local/lib/python3.7/site-packages/click/core.py", line 1137, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/usr/local/lib/python3.7/site-packages/click/core.py", line 1137, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/usr/local/lib/python3.7/site-packages/click/core.py", line 956, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/usr/local/lib/python3.7/site-packages/click/core.py", line 555, in invoke
    return callback(*args, **kwargs)
  File "/usr/local/lib/python3.7/site-packages/shodan/cli/data.py", li

## Listing the data within 'raw-daily' dataset (and saving it in a file)

In [19]:
!shodan data list --dataset=raw-daily > shodan_list_daily_files.txt

In [20]:
!cat shodan_list_daily_files.txt

2020-06-23.json.gz  114.2 GB  43f00025cf9d2f2afe60b4b33bf38858df40b8b3  https://f001.backblazeb2.com/b2api/v1/b2_download_file_by_id?fileId=4_ze3b56ca7daafe60052c90a15_f21096ed7e0e8da28_d20200624_m015713_c001_v0001128_t0002
2020-06-22.json.gz  114.0 GB  96e7bb71c06583e3b291435be8aba162bcf5c974  https://f001.backblazeb2.com/b2api/v1/b2_download_file_by_id?fileId=4_ze3b56ca7daafe60052c90a15_f2092b2ff74613b40_d20200623_m015541_c001_v0001137_t0041
2020-06-21.json.gz  103.3 GB  a41fd520cfbe4e3f211d3247396975d33d2a5acc  https://f001.backblazeb2.com/b2api/v1/b2_download_file_by_id?fileId=4_ze3b56ca7daafe60052c90a15_f201b79977f7b364e_d20200622_m014812_c001_v0001137_t0049
2020-06-20.json.gz  1.8 GB    3855f518fb2ca6655f7ccd04f2954264fd2c6e47  https://f001.backblazeb2.com/b2api/v1/b2_download_file_by_id?fileId=4_ze3b56ca7daafe60052c90a15_f21096ed7e0e7785a_d20200621_m003120_c001_v0001137_t0044
2020-06-19.json.gz  105.3 GB  b72bb9e248dd6b3f41138f9b29904733830c9107  https://f001.backblazeb2.com/b2a

**NOTE:** As you can see the data available contains only the last 30 days. Each day has around 100G of data. Each entry contains the direct link to the data (something like https://f001.backblazeb2.com/b2api/v1/b2_download_file_by_id?fileId=...). It looks like that **anyone with the link can download the data.** 100G times 30 days is around 3T, it will take ages for downloading it. Following we played a bit with Aria2 for downloading things in parallel.

<h1 align='center'>========================================================</h1>

## Trying to download 1 file via Aria2c (https://aria2.github.io/) for parallel threads download using command line

**NOTE:** you must install 'aria2' because 'aria2c' is the command-line tool.

**HOW TO USE:**

!aria2c -x<"NUMBER_CONNECTIONS_PER_HOST"> -s<"NUMBER_THREADS"> -o <"OUTPUT_FILE_NAME"> <"DIRECT_DOWNLOAD_LINK">

**EXAMPLE:**

!aria2c -x4 -s4 -o shodan_raw_2019-08-14.json.gz https://f001.backblazeb2.com/b2api/v1/b2_download_file_by_id?fileId=4_ze3b56ca7daafe60052c90a15_f2036e4bb0915da7f_d20190815_m000003_c001_v0001092_t0019

In [ ]:
!aria2c -x4 -s4 -o shodan_raw_2019-08-14.json.gz https://f001.backblazeb2.com/b2api/v1/b2_download_file_by_id?fileId=4_ze3b56ca7daafe60052c90a15_f2036e4bb0915da7f_d20190815_m000003_c001_v0001092_t0019

<h1 align='center'>========================================================</h1>

## Automatic downloading all the data listed in 'raw-daily' as performed followed by the preprocessing data. I describe the script in the last session.

<h1 align='center'>========================================================</h1>

## (Offline) pre-processing the raw data
Each file (example: 'raw-daily-2019-07-17.json.gz') contains *unstructured data*. It is a .json file, each line is 
we need to (1) extract only the lines that contain in any field the word "mikrotik" (save the output in a file: "mikrotik"), then use these mikortik-related entries for (2) extract only a couple of fields of our interest (i.e., IP address, port number, country code, version, and product) (save the output in another file: "simplified"). No judgments on "throwing away" data, okey?! Remember it will be more then 1M entries to pos-process.

## 0st. Defining which downloaded raw file would you like to pre-process

In [ ]:
shodan_raw_filename='raw-daily-2019-07-17.json.gz'

## 1st. Getting only lines that have the word 'mikrotik' 

In [ ]:
shodan_mikrotik_filename=shodan_raw_filename.replace('raw-daily','mikrotik')

In [ ]:
!zgrep -i 'mikrotik' $shodan_raw_filename |gzip > $shodan_mikrotik_filename

**NOTE:** zgrep is faster than any python code!

## 2nd. Simplifying the (already filtered) .json file to get only a set of fields
**NOTE:** Ofcourse, YES, we've tried 'jq' [(https://stedolan.github.io/jq/)](https://stedolan.github.io/jq/) first BUT it didn't work!

In [ ]:
import json
import gzip

outputfile_name = shodan_mikrotik_filename.split('.')[0]+'_simplified.json'

outputfile = open(outputfile_name, 'w')

data=[]
for line in gzip.open(shodan_mikrotik_filename, 'r'):
    loaded_line = json.loads(line)
    try:
        ip = loaded_line['ip_str']
    except:
        ip = ""
    try:
        port = loaded_line['port']
    except:
        port = ""
    try:
        cc = loaded_line['location']['country_code3']
    except:
        cc = ""
    try:
        version = loaded_line['version']
    except:
        version = ""    
    try:
        product = loaded_line['product']
    except:
        product = ""
        
    json.dump({'ip':ip,
               'port':port,
               'cc': cc,
               'version':version,
               'product': product}, outputfile)
    
    outputfile.write("\n")

outputfile.close()

## 4th. [ BONUS] Sending an email to notify that the script ended!
This is the lazy way to leave a code running and returning when it is done =D

In [ ]:
def send_email():
    import smtplib

    gmail_user = 'jjsantanna.script@gmail.com'
    gmail_password = '<PASSWORD-HERE>'

    sent_from = gmail_user
    to = ['j.j.santanna@utwente.nl']
    subject = 'Script Ended Bitch'
    body = "This is an automatic message!\nGo to your computer and check it out."

    email_text = "From: %s\nTo: %s\nSubject: %s\n\n%s\n" % (sent_from, ", ".join(to), subject, body)

    try:
        server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
        server.ehlo()
        server.login(gmail_user, gmail_password)
        server.sendmail(sent_from, to, email_text)
        server.close()

        print ('Email sent!')
    except:
        print ('Something went wrong...')

In [ ]:
send_email()

<h1 align='center'>========================================================</h1>

## EXAMPLE of OFFLINE analyzing a simplified raw data from Shodan

In [22]:
import json
import pandas as pd
import matplotlib as mpl

## Reading the json file
**NOTE:** pandas.read_json() didn't work. This is why we did a workaround. 

In [23]:
data = [json.loads(line) for line in open('data_example/shodan_mikrotik_2019-08-14_simplified.json', 'r')]
df = pd.DataFrame(data)

In [6]:
len(df)

599082

In [8]:
df.columns

Index(['cc', 'ip', 'port', 'product', 'version'], dtype='object')

In [10]:
df['port'].value_counts().head()

2000    358893
1723    117511
80       32607
21       25959
8080     15967
Name: port, dtype: int64

<h1 align='center'>========================================================</h1>

At the end we wrote a bash script to do everything for us!

## Bash script to (1) get a list of Shodan.io raw files, (2) download each file, (3) pre-process each file, (4) simplify the pre-processed file + send email to me, and (5) move the file to cold storage.

```bash
#!/bin/bash

echo "Please enter your Shodan API key, followed by [ENTER]:"
read shodankey

shodan init $shodankey

shodan data list --dataset=raw-daily > shodan_list_daily_files.txt

cat shodan_list_daily_files.txt | while read filename filesize id link
do 
	echo $filename
	aria2c -x4 -s4 -o shodan_raw_$filename $link
	zgrep -i 'mikrotik' shodan_raw_$filename| gzip > shodan_mikrotik_$filename
	python3 simplify_json_gz.py -i shodan_mikrotik_$filename
	mv $filename /Volumes/LaCie
done
```